# Reranking search results with cross-encoders

This notebook takes you through examples of using a cross-encoder to re-rank search results.

This is a common use case with our customers, where you've implemented semantic search using embeddings (produced using a [Bi-encoder](https://www.sbert.net/examples/applications/retrieve_rerank/README.html#retrieval-bi-encoder)) but the results are not exactly what you need. A possible cause is that there is some business rule you can use to rerank the documents such as how recent or how popular a document is. 

However, often there are subtle domain-specific rules that help determine relevancy, and this is where a cross-encoder can be useful. Cross-encoders are more accurate than bi-encoders but they don't scale well, so using them to re-order a shortened list returned by semantic search is the ideal use case.

We'll use text-davinci-003 with logprobs to build a cross-encoder here.

This notebook drew on this great [article](https://weaviate.io/blog/cross-encoders-as-reranker) by Weaviate, and this [excellent explanation](https://www.sbert.net/examples/applications/cross-encoder/README.html) of bi-encoders vs. cross-encoders from Sentence Transformers.

In [ ]:
!pip install arxiv
!pip install tenacity
!pip install pandas

In [89]:
import arxiv
from math import exp
import openai
import pandas as pd
from tenacity import retry, wait_random_exponential, stop_after_attempt

## Search

We'll use the arXiv search service for this example, but this step could be performed by any search service you have. The key item to consider is over-fetching slightly to capture all the potentially relevant documents, before re-sorting them.

To illustrate our point we'll sort by "SubmittedDate" rather than "Relevance" so we get some irrelevant results in the top 5.

In [75]:
query = "how do bi-encoders work for sentence embeddings"
search = arxiv.Search(
    query=query, max_results=10, sort_by=arxiv.SortCriterion.SubmittedDate
)

In [76]:
result_list = []

for result in search.results():
    result_dict = {}

    result_dict.update({"title": result.title})
    result_dict.update({"summary": result.summary})

    # Taking the first url provided
    result_dict.update({"article_url": [x.href for x in result.links][0]})
    result_dict.update({"pdf_url": [x.href for x in result.links][1]})
    result_list.append(result_dict)

In [77]:
result_list[0]

{'title': 'ModuleFormer: Learning Modular Large Language Models From Uncurated Data',
 'summary': 'Large Language Models (LLMs) have achieved remarkable results. But existing\nmodels are expensive to train and deploy, and it is also difficult to expand\ntheir knowledge beyond pre-training data without forgetting previous knowledge.\nThis paper proposes a new neural network architecture, ModuleFormer, that\nleverages modularity to improve the efficiency and flexibility of large\nlanguage models. ModuleFormer is based on the Sparse Mixture of Experts (SMoE).\nUnlike the previous SMoE-based modular language model [Gururangan et al.,\n2021], which requires domain-labeled data to learn domain-specific experts,\nModuleFormer can induce modularity from uncurated data with its new load\nbalancing and load concentration losses. ModuleFormer is a modular architecture\nthat includes two different types of modules, new stick-breaking attention\nheads, and feedforward experts. Different modules are

In [78]:
for i, result in enumerate(result_list):
    print(f"{i + 1}: {result['title']}")

1: ModuleFormer: Learning Modular Large Language Models From Uncurated Data
2: Lopsidedness as a tracer of early galactic assembly history
3: Transformers as Statisticians: Provable In-Context Learning with In-Context Algorithm Selection
4: GP-UNIT: Generative Prior for Versatile Unsupervised Image-to-Image Translation
5: A New Paradigm Unifying the Concepts in Particle Abrasion and Breakage
6: On the Reliability of Watermarks for Large Language Models
7: Contrastive Lift: 3D Object Instance Segmentation by Slow-Fast Contrastive Fusion
8: Designing a Better Asymmetric VQGAN for StableDiffusion
9: On P^1-stabilization in unstable motivic homotopy theory
10: One-loop Integrals from Volumes of Orthoschemes


## Cross-encoder

We'll create a cross-encoder using the ```Completions``` endpoint - the key factors to consider here are:
- Make your examples domain-specific - the strength of cross-encoders comes when you fine-tune them to your domain.
- There is a trade-off between how many potential examples to re-rank vs. processing speed. Consider batching and parallel processing cross-encoder requests to aid user experience.

The steps here are:
- Build a prompt to assess relevance and provide few-shot examples to tune it to your domain
- Return the classification of yes/no as well as the ```logprobs```
- Rerank the results by the ```logprobs``` keyed on Yes

In [79]:
prompt = '''
You are an Assistant responsible for helping detect whether the retrieved document is relevant to the query. For a given input, you need to output a single token: "Yes" or "No" indicating the retrieved document is relevant to the query.

Examples:
"""
Query: How to plant a tree?
Retrieved document: Cars were invented in 1886, when German inventor Carl Benz patented his Benz Patent-Motorwagen.[3][4][5] Cars became widely available during the 20th century. One of the first cars affordable by the masses was the 1908 Model T, an American car manufactured by the Ford Motor Company. Cars were rapidly adopted in the US, where they replaced horse-drawn carriages.[6] In Europe and other parts of the world, demand for automobiles did not increase until after World War II.[7] The car is considered an essential part of the developed economy.
Relevant to the query?
Assistant: No

Query: Has the coronavirus vaccine been approved?
Retrieved document: The Pfizer-BioNTech COVID-19 vaccine was approved for emergency use in the United States on December 11, 2020.
Relevant to the query?
Assistant: Yes

Query: What is the capital of France?
Retrieved document: Paris, France's capital, is a major European city and a global center for art, fashion, gastronomy and culture. Its 19th-century cityscape is crisscrossed by wide boulevards and the River Seine. Beyond such landmarks as the Eiffel Tower and the 12th-century, Gothic Notre-Dame cathedral, the city is known for its cafe culture and designer boutiques along the Rue du Faubourg Saint-Honoré. 
Relevant to the query?
Assistant: Yes

Query: What are some papers to learn about PPO reinforcement learning?
Retrieved document: Proximal Policy Optimization and its Dynamic Version for Sequence Generation: In sequence generation task, many works use policy gradient for model optimization to tackle the intractable backpropagation issue when maximizing the non-differentiable evaluation metrics or fooling the discriminator in adversarial learning. In this paper, we replace policy gradient with proximal policy optimization (PPO), which is a proved more efficient reinforcement learning algorithm, and propose a dynamic approach for PPO (PPO-dynamic). We demonstrate the efficacy of PPO and PPO-dynamic on conditional sequence generation tasks including synthetic experiment and chit-chat chatbot. The results show that PPO and PPO-dynamic can beat policy gradient by stability and performance.
Relevant to the query?
Assistant: Yes

Query: Explain sentence embeddings
Retrieved document: Inside the bubble: exploring the environments of reionisation-era Lyman-α emitting galaxies with JADES and FRESCO: We present a study of the environments of 16 Lyman-α emitting galaxies (LAEs) in the reionisation era (5.8<z<8) identified by JWST/NIRSpec as part of the JWST Advanced Deep Extragalactic Survey (JADES). Unless situated in sufficiently (re)ionised regions, Lyman-α emission from these galaxies would be strongly absorbed by neutral gas in the intergalactic medium (IGM). We conservatively estimate sizes of the ionised regions required to reconcile the relatively low Lyman-α velocity offsets (ΔvLyα<300kms−1) with moderately high Lyman-α escape fractions (fesc,Lyα>5%) observed in our sample of LAEs, indicating the presence of ionised ``bubbles'' with physical sizes of the order of 0.1pMpc≲Rion≲1pMpc in a patchy reionisation scenario where the bubbles are embedded in a fully neutral IGM. Around half of the LAEs in our sample are found to coincide with large-scale galaxy overdensities seen in FRESCO at z∼5.8-5.9 and z∼7.3, suggesting Lyman-α transmission is strongly enhanced in such overdense regions, and underlining the importance of LAEs as tracers of the first large-scale ionised bubbles. Considering only spectroscopically confirmed galaxies, we find our sample of UV-faint LAEs (MUV≳−20mag) and their direct neighbours are generally not able to produce the required ionised regions based on the Lyman-α transmission properties, suggesting lower-luminosity sources likely play an important role in carving out these bubbles. These observations demonstrate the combined power of JWST multi-object and slitless spectroscopy in acquiring a unique view of the early stages of Cosmic Reionisation via the most distant LAEs.
Relevant to the query?
Assistant: No
"""
End of Examples

Begin!

Query: {query}
Retrieved document: {document}
Relevant to the query?
Assistant:
'''


@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def rerank_document(query, document, output_list):
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt.format(query=query, document=content),
        temperature=0,
        logprobs=1,
    )

    output_list.append(
        (
            query,
            document,
            response["choices"][0]["text"],
            response["choices"][0]["logprobs"]["token_logprobs"][0],
        )
    )

In [80]:
content = result_list[0]["title"] + ": " + result_list[0]["summary"]

# Set logprobs to 1 so we return the most likely token with the likelihood
response = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt.format(query=query, document=content),
    temperature=0,
    logprobs=1,
)

In [81]:
result = response["choices"][0]
print(f"Result was {result['text']}")
print(f"Logprobs was {result['logprobs']['token_logprobs'][0]}")
print("\nBelow is the full logprobs object\n\n")
print(result["logprobs"])

Result was Yes
Logprobs was -0.0013786046

Below is the full logprobs object


{
  "text_offset": [
    6216
  ],
  "token_logprobs": [
    -0.0013786046
  ],
  "tokens": [
    "Yes"
  ],
  "top_logprobs": [
    {
      "Yes": -0.0013786046
    }
  ]
}


In [82]:
output_list = []
for x in result_list:
    content = x["title"] + ": " + x["summary"]

    try:
        rerank_document(query, document=content, output_list=output_list)

    except Exception as e:
        print(e)

In [86]:
output_list[:3]

[('how do bi-encoders work for sentence embeddings',
  'ModuleFormer: Learning Modular Large Language Models From Uncurated Data: Large Language Models (LLMs) have achieved remarkable results. But existing\nmodels are expensive to train and deploy, and it is also difficult to expand\ntheir knowledge beyond pre-training data without forgetting previous knowledge.\nThis paper proposes a new neural network architecture, ModuleFormer, that\nleverages modularity to improve the efficiency and flexibility of large\nlanguage models. ModuleFormer is based on the Sparse Mixture of Experts (SMoE).\nUnlike the previous SMoE-based modular language model [Gururangan et al.,\n2021], which requires domain-labeled data to learn domain-specific experts,\nModuleFormer can induce modularity from uncurated data with its new load\nbalancing and load concentration losses. ModuleFormer is a modular architecture\nthat includes two different types of modules, new stick-breaking attention\nheads, and feedforward

In [87]:
output_df = pd.DataFrame(
    output_list, columns=["query", "document", "prediction", "logprobs"]
).reset_index()
# Use exp() to convert logprobs into probability
output_df["probability"] = output_df["logprobs"].apply(exp)
# Reorder based on likelihood of being Yes
output_df["cleaned_probability"] = output_df.apply(
    lambda x: x["probability"] * -1 + 1
    if x["prediction"] == "No"
    else x["probability"],
    axis=1,
)
output_df.head()

,index,query,document,prediction,logprobs,probability,cleaned_probability
0,0,how do bi-encoders work for sentence embeddings,ModuleFormer: Learning Modular Large Language ...,Yes,-0.001388,0.998613,0.998613
1,1,how do bi-encoders work for sentence embeddings,Lopsidedness as a tracer of early galactic ass...,No,-0.000833,0.999168,0.000832
2,2,how do bi-encoders work for sentence embeddings,Transformers as Statisticians: Provable In-Con...,Yes,-0.001508,0.998493,0.998493
3,3,how do bi-encoders work for sentence embeddings,GP-UNIT: Generative Prior for Versatile Unsupe...,No,-0.000779,0.999221,0.000779
4,4,how do bi-encoders work for sentence embeddings,A New Paradigm Unifying the Concepts in Partic...,No,-0.000825,0.999175,0.000825


In [88]:
# Return reranked results
output_df.sort_values(by=["cleaned_probability"], ascending=False)

,index,query,document,prediction,logprobs,probability,cleaned_probability
0,0,how do bi-encoders work for sentence embeddings,ModuleFormer: Learning Modular Large Language ...,Yes,-0.001388,0.998613,0.998613
2,2,how do bi-encoders work for sentence embeddings,Transformers as Statisticians: Provable In-Con...,Yes,-0.001508,0.998493,0.998493
8,8,how do bi-encoders work for sentence embeddings,On P^1-stabilization in unstable motivic homot...,No,-0.000883,0.999117,0.000883
7,7,how do bi-encoders work for sentence embeddings,Designing a Better Asymmetric VQGAN for Stable...,No,-0.000837,0.999164,0.000836
1,1,how do bi-encoders work for sentence embeddings,Lopsidedness as a tracer of early galactic ass...,No,-0.000833,0.999168,0.000832
4,4,how do bi-encoders work for sentence embeddings,A New Paradigm Unifying the Concepts in Partic...,No,-0.000825,0.999175,0.000825
3,3,how do bi-encoders work for sentence embeddings,GP-UNIT: Generative Prior for Versatile Unsupe...,No,-0.000779,0.999221,0.000779
6,6,how do bi-encoders work for sentence embeddings,Contrastive Lift: 3D Object Instance Segmentat...,No,-0.000775,0.999225,0.000775
9,9,how do bi-encoders work for sentence embeddings,One-loop Integrals from Volumes of Orthoscheme...,No,-0.000648,0.999352,0.000648
5,5,how do bi-encoders work for sentence embeddings,On the Reliability of Watermarks for Large Lan...,No,-0.000585,0.999415,0.000585


## Consolidation

We've used the ```Completions``` endpoint from OpenAI to build our cross-encoder, but this area is well-served by the open-source community. [Here](https://huggingface.co/cross-encoder/mmarco-mMiniLMv2-L12-H384-v1) is an example from HuggingFace, for example.

We hope you find this useful for tuning your search use cases, and look forward to seeing what you build.